#  DFFRAM Compiler


 Standard Cell Library based Memory Compiler using DFF/Latch cells.





### Overview
DFFRAM is based around two Python modules: the DFFRAM flow and the placeram utility.

placeram is a custom placer using OpenROAD's Python interface. It places DFFRAM RAM/register file designs in a predetermined structure to avoid a lengthy and inefficient manual placement process for RAM. 

dffram.py is a relatively self-contained flow that uses placeram and OpenLane to place, route and harden RAM. 

### Supported Sizes 

RAM: 
- 32 words with byte write enable (1RW and 1RW1R).
- 128 words with byte write enable (1RW and 1RW1R).
- 256 words with byte write enable (1RW and 1RW1R).
- 512 words with byte write enable (1RW and 1RW1R).
- 1024 words with byte write enable (1RW and 1RW1R).
- 2048 words with byte write enable (1RW and 1RW1R).


RF: 

- 32 x 32-bit words (2R1W)



---



### Installing Conda Package Manager for Google Colab

In [ ]:
!pip install -q condacolab
import condacolab

condacolab.install_from_url(
    "https://repo.anaconda.com/miniconda/Miniconda3-py37_4.11.0-Linux-x86_64.sh"
)

condacolab.check()
!sed -i -e /cudatoolkit/d /usr/local/conda-meta/pinned

The litex-hub conda repository maintains a collection of EDA tools- including those needed to support OpenLane and DFFRAM.

In [ ]:
!conda install -y -c litex-hub -c conda-forge openroad=2.0_4703_g6a7ef2a6d  magic netgen yosys gdstk

### Cloning OpenLane and DFFRAM and Installing the required dependencies:  🔄

(We're also setting some environment variables that make OpenLane and DFFRAM play nicer with conda.)

In [ ]:
!apt-get update && apt-get install -yq tcllib
%env TCLLIBPATH=/usr/share/tcltk

%cd /content
!rm -rf ./openlane
!mkdir -p ./openlane
!curl -L https://github.com/The-OpenROAD-Project/OpenLane/tarball/57f3f21d6e1d0403a88d7a7973f7447597aae160 | tar -xzC ./openlane --strip-components=1
!python3 -m pip install -r /content/openlane/requirements.txt

%cd /content
!rm -rf ./dffram
!mkdir -p ./dffram
!curl -L https://github.com/AUCOHL/DFFRAM/tarball/5e7e9674a1373ddb65445b4985488adc154efcd9 | tar -xzC ./dffram --strip-components=1
!python3 -m pip install -r /content/dffram/requirements.txt


%cd /content/dffram
!volare enable --pdk-root /content/dffram/pdk --pdk sky130 44a43c23c81b45b8e774ae7a84899a5a778b6b0b

%env PDK=sky130A
%env QUIT_ON_MISMATCHES=0
%env NO_CHECK_INSTALL=1



---



### For a full list of different options 🤔  

In [ ]:
!./dffram.py --help

### 32x32 with one read-write port and one read-only port 🚀 

In [ ]:
!./dffram.py --using-local-openlane /content/openlane -s 32x32

### 32x32 with two read ports and one write port (a register file)

In [ ]:
!./dffram.py --using-local-openlane /content/openlane -s 32x32 -b ::rf -v 2R1W

### Aaand visualize it! 🎉

In [ ]:
import pathlib
import gdstk
from IPython.display import SVG

gds = sorted(
    pathlib.Path("/content/DFFRAM/build/32x32_DEFAULT").glob("*/gds/RAM32.gds")
)[0]
library = gdstk.read_gds(gds)
top_cells = library.top_level()
top_cells[0].write_svg("RAM32.svg")
SVG("RAM32.svg")